In [109]:
import pandas as pd
from gurobipy import Model, GRB

In [110]:
# Load the non-profit data from the CSV file
df = pd.read_csv(r"C:\Users\johns\OneDrive\Desktop\MBAN Semester 3\OMIS 6000 - Models & Applications in Operational Research\Midterm\non_profits.csv")

In [111]:
alpha = df['alpha_i'].tolist()
beta = df['beta_i'].tolist()

In [112]:
# Set up the optimization model
model = Model('NonProfitFunding')

In [113]:
# Define the budget
budget = 50e6  # $50 million

In [114]:
# Define the decision variables for allocations
a = model.addVars(len(df), lb=0, name='Allocation')

In [115]:
# Create auxiliary variables for the product of a[i] and beta_i
product_vars = model.addVars(len(df), name='product', lb=0)

# Create auxiliary variables to represent (product)^1/3
aux_vars = model.addVars(len(df), name='aux', lb=0)

# Create variables for the squared term (output)
output_vars = model.addVars(len(df), name='output', lb=0)

In [116]:
# Constraint for the budget
model.addConstr(a.sum() <= budget, 'Budget')

# Constraints that define the product of allocation and beta_i
for i in range(len(df)):
    model.addConstr(product_vars[i] == a[i] * df['beta_i'][i], name=f'product_{i}')

In [117]:
# Add power constraints using addGenConstrPow
for i in range(len(df)):
    model.addGenConstrPow(product_vars[i], aux_vars[i], 1/3, name=f'Power_{i}')

In [118]:
# Constraints that define the output as the square of the aux_vars
for i in range(len(df)):
    model.addConstr(output_vars[i] == aux_vars[i] * aux_vars[i], name=f'output_{i}')

In [119]:
# Set the objective to maximize the total output
model.setObjective(2 * output_vars.sum(), GRB.MAXIMIZE)

In [120]:
# Optimize the model
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads



GurobiError: Model too large for size-limited license; visit https://www.gurobi.com/free-trial for a full license

In [121]:
# Output the optimal allocations and the total expected output
if model.status == GRB.OPTIMAL:
    total_output = model.objVal
    allocated_budget = sum(a[i].x for i in range(len(df)))
    print(f'Total Expected Output: {total_output}')
    for i in range(len(df)):
        if a[i].x > 0:
            print(f'Non-profit {i}: Allocation = {a[i].x}')
else:
    print('No optimal solution found.')

No optimal solution found.
